In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from model.Base import Base
from model.DeviceAppTraffic import DeviceAppTraffic

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

#output_notebook()

DB='postgresql+psycopg2:///ucnstudy'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine) 

In [3]:
ses = Session()

devApp = ses.query(DeviceAppTraffic).limit(10)

for item in devApp:
    print (item.bytes_in)

408
5033
387
156
177
266
204
60
148
142
